In [51]:
import numpy as np
import os
import fnmatch
import csv
from scipy.spatial.distance import cdist
from PIL import Image, ImageDraw, ImageFont
from matplotlib import colors

In [81]:
dict_data = {}
templates = [] 
test = []

for path,dirs,files in os.walk('.') : 
    for f in fnmatch.filter(files,'*.csv') : 
        fullname = os.path.abspath(os.path.join(path,f))
        utterance = [] 
        code = (fullname.split('/')[7]+'_'+fullname.split('/')[8]).split('.')[0]
        
        if int(fullname.split('/')[8].split('.')[0]) <=3 : 
            templates.append(code)
        else :
            test.append(code)
            
        with open(fullname,'r') as segment:
            read = csv.reader(segment)
            for row in read : 
                utterance.append([float(row[0].split()[0]),float(row[0].split()[1])])
        np_utterance = np.array(utterance)
        np_utterance = np_utterance.reshape(-1,1024,2)
        dict_data[code] = np_utterance  
        print(fullname)

/home/saik/Semester 6/CS6300/Assignment 6/Cepstrum_15ms_10ms/7_1/5.csv
/home/saik/Semester 6/CS6300/Assignment 6/Cepstrum_15ms_10ms/7_1/3.csv
/home/saik/Semester 6/CS6300/Assignment 6/Cepstrum_15ms_10ms/7_1/2.csv
/home/saik/Semester 6/CS6300/Assignment 6/Cepstrum_15ms_10ms/7_1/1.csv
/home/saik/Semester 6/CS6300/Assignment 6/Cepstrum_15ms_10ms/7_1/4.csv
/home/saik/Semester 6/CS6300/Assignment 6/Cepstrum_15ms_10ms/15_1/5.csv
/home/saik/Semester 6/CS6300/Assignment 6/Cepstrum_15ms_10ms/15_1/3.csv
/home/saik/Semester 6/CS6300/Assignment 6/Cepstrum_15ms_10ms/15_1/2.csv
/home/saik/Semester 6/CS6300/Assignment 6/Cepstrum_15ms_10ms/15_1/1.csv
/home/saik/Semester 6/CS6300/Assignment 6/Cepstrum_15ms_10ms/15_1/4.csv
/home/saik/Semester 6/CS6300/Assignment 6/Cepstrum_15ms_10ms/16_1/5.csv
/home/saik/Semester 6/CS6300/Assignment 6/Cepstrum_15ms_10ms/16_1/3.csv
/home/saik/Semester 6/CS6300/Assignment 6/Cepstrum_15ms_10ms/16_1/2.csv
/home/saik/Semester 6/CS6300/Assignment 6/Cepstrum_15ms_10ms/16_1/1.c

In [4]:
def DTW2(template,inp):
    r, c = len(template), len(inp)
    X=np.zeros((r,2048))
    Y=np.zeros((c,2048))

    for i in range(r):
        X[i,:]=np.append(template[i,:,0],template[i,:,1])
    for i in range(c):
        Y[i,:]=np.append(inp[i,:,0],inp[i,:,1])
    
    dtw = np.zeros((r , c))
    dtw= cdist(X, Y, 'sqeuclidean')
    
    for i in range(r):
        for j in range(c):
            if np.isnan(dtw[i][j]) : 
                dtw[i][j] = 0
    
    for i in range(r):
        for j in range(c):
            if(i==0 and j!=0):
                min_list = [dtw[i,j-1]]
                dtw[i, j] += min(min_list)
            elif(i==1 and j!=0):
                min_list = [dtw[i-1, j-1],dtw[i,j-1]]
                dtw[i, j] += min(min_list)
            elif(j!=0):
                min_list = [dtw[i-2, j-1],dtw[i-1, j-1],dtw[i,j-1]]
                dtw[i, j] += min(min_list)
    return dtw[-1,-1]/c

In [82]:
a = 0
confusion_matrix=np.zeros((23,23))
def scoring() : 
    test_scores = {}
    values_score = []
    key_scores = {}
    for j in templates : 
        test_scores[j] = DTW2(dict_data[j],dict_data[test[a]])
        values_score.append(DTW2(dict_data[j],dict_data[test[a]]))  
    values_score.sort()
    return values_score,test_scores

accuracy = 0

for a in range(len(test)) :
    values_score,test_scores = scoring()
    print('Test : ',test[a])
    b=''
    for key in test_scores.keys() : 
        if test_scores[key] == values_score[0] :
            print(key , values_score[0])
            if key[0:4] == test[a][0:4] :
                print('success')
                accuracy = accuracy + 1
            else : 
                print('Failed')
            b=key[0:4]
            break
    confusion_matrix[list_folder.index(b)][list_folder.index(test[a][0:4])]+=1        
print('Accuracy : ',accuracy/len(test))   

Test :  7_1_5
7_1_3 643969.3946294592
success
Test :  7_1_4
7_1_3 602872.0783688168
success
Test :  15_1_5
15_2_3 761114.1406620363
Failed
Test :  15_1_4
15_1_2 729389.3588380328
success
Test :  16_1_5
16_1_3 537847.3994228583
success
Test :  16_1_4
16_1_3 533604.2108594431
success
Test :  9_1_5
9_1_1 604310.0153734245
success
Test :  9_1_4
9_1_1 595153.8328288194
success
Test :  1_1_5
1_1_3 588026.2996943484
success
Test :  1_1_4
1_1_3 600125.894994707
success
Test :  19_1_5
19_1_2 554111.5405503832
success
Test :  19_1_4
18_1_1 553782.0640452922
Failed
Test :  6_2_5
6_2_2 642526.2253366113
success
Test :  6_2_4
6_2_3 632700.0100807248
success
Test :  19_2_5
18_2_2 552018.0731291362
Failed
Test :  19_2_4
19_2_3 560388.2419130002
success
Test :  4_2_5
4_2_1 569952.3052244063
success
Test :  4_2_4
4_2_3 698966.0095672058
success
Test :  8_1_5
8_1_3 628273.9678154922
success
Test :  8_1_4
18_2_3 12966356.866013583
Failed
Test :  6_1_5
6_1_2 646260.2576601006
success
Test :  6_1_4
6_1_3 6

In [83]:
def imagegenerator(cm):
    step_count = 23
    height = 690
    width = 690
    image = Image.new('RGBA', (height, width), 'white')

    draw = ImageDraw.Draw(image)
    y_start = 0
    y_end = image.height
    step_size = int(image.width / step_count)

    for x in range(0, image.width, step_size):
        line = ((x, y_start), (x, y_end))
        draw.line(line, fill=128)

    x_start = 0
    x_end = image.width

    for y in range(0, image.height, step_size):
        line = ((x_start, y), (x_end, y))
        draw.line(line, fill=128)

    for i in range(23):
        for j in range(23):
            string=str(cm[i,j])
            if(cm[i,j]==2 and i==j):
                draw.rectangle((30*i, 30*j, 30*(i+1), 30*(j+1)), outline='black', fill='green')  
            elif(cm[i,j]==1 and i==j):
                draw.rectangle((30*i, 30*j, 30*(i+1), 30*(j+1)), outline='black', fill='blue') 
            elif(cm[i,j]!=0):
                draw.rectangle((30*i, 30*j, 30*(i+1), 30*(j+1)), outline='black', fill='red') 
            draw.text((30*i+10,30*j+10),string, fill='black')

    del draw

    image.show()

imagegenerator(np.transpose(confusion_matrix))